In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.linear_model import Lasso,Ridge
import os
print(os.listdir("../input"))
import lightgbm as lgb
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn import metrics
import time
from sklearn.svm import LinearSVR
# Any results you write to the current directory are saved as output.

In [ ]:
# 把catboost,lightgbm,nn,xgb的验证集输出做交叉验证的训练集
cat_oof=pd.read_csv('../input/catboost-based-method/cat_oof.csv')['oof']
lgb_oof=pd.read_csv('../input/lightgbm-based-model/lgb_oof.csv')['oof']
nn_oof=pd.read_csv('../input/nn-based-method/nn_oof.csv')['oof']
xgb_oof=pd.read_csv('../input/xgboost-based-model/xgb_oof.csv')['oof']

# 读取y
y_0 = pd.read_csv("../input/feature1/train_y.csv", index_col=False,  header=None)
y_1 = pd.read_csv("../input/feature2/train_y1.csv", index_col=False,  header=None)
y = pd.concat([y_0, y_1], axis=0)
y = y.reset_index(drop=True)

# catboost,lightgbm,nn,xgb的预测结果
cat_pred=pd.read_csv('../input/catboost-based-method/CatBoost_submission_1.242.csv')['time_to_failure']
nn_pred=pd.read_csv('../input/nn-based-method/NN_submission_1.355.csv')['time_to_failure']
lgb_pred=pd.read_csv('../input/lightgbm-based-model/lgb_submission_0.929.csv')['time_to_failure']
xgb_pred=pd.read_csv('../input/xgboost-based-model/xgb_submission_1.031.csv')['time_to_failure']

In [ ]:
cat_oof.head()

In [ ]:
nn_oof.head()

In [ ]:
xgb_oof.head()

In [ ]:
#把cat_oof,xgb_oof,lgb_oof,nn_oof合成训练集
train_stack = np.vstack([cat_oof,xgb_oof,lgb_oof,nn_oof]).transpose()
train_stack = pd.DataFrame(train_stack, columns=['cat','xgb','lgb','nn'])
train_stack.shape

#把 cat_pred,xgb_pred,lgb_pred,nn_pred合成测试集
test_stack = np.vstack([cat_pred,xgb_pred,lgb_pred,nn_pred]).transpose()
test_stack = pd.DataFrame(test_stack, columns=['cat','xgb','lgb','nn'])


In [ ]:
# 构造stacking模型
model=Ridge()
model.fit(train_stack,y)
r1=model.predict(test_stack).reshape(-1)

In [ ]:
# 将结果输出
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission['time_to_failure'] =r1
print(submission.head())
submission.to_csv('stacking.csv',index=False)
